In [ ]:
import pandas as pd
import json
from collections import defaultdict
import re
import datetime, time

#### Con este bucle obtenemos los datos sobre la inicialización de todos los niveles jugados

In [ ]:
def extraerTiempos(rawData):
    erLevel = re.compile(r'\blevel$\b')
    erInitialized = re.compile(r'\binitialized$\b')
    erCompleted = re.compile(r'\bcompleted$\b')
    erIdLevel = re.compile(r'/')
    
    data = defaultdict(defaultdict)
    anomalias = []

    for evento in rawData:
        verb = evento["verb"]["id"]
        obj = evento["object"]["definition"]["type"]
        if erLevel.search(obj) : #Si el objeto de la acción es un nivel
            name = evento["actor"]["name"]
            levelCode = erIdLevel.split(evento["object"]["id"])[-1]
            timestamp = evento["timestamp"]
            if erInitialized.search(verb): #Si la acción es inicio o reinicio
                if "result" in evento: #Significa que ha iniciado el nivel desde el menu
                    if levelCode in data[name]:
                        data[name][levelCode].append({"ini" : timestamp, "fin" : "", "stars" : ""})
                    else:
                        data[name][levelCode] = [{"ini" : timestamp, "fin" : "", "stars" : ""}]
                #else: #-> sería cuando un nivel es reiniciado, por ahora no vamos a tenerlo en cuenta
            elif erCompleted.search(verb):
                if evento["result"]["success"]: #Significa que el nivel se ha completado con éxito
                    if levelCode in data[name]:
                        data[name][levelCode][-1]["fin"] = timestamp
                        data[name][levelCode][-1]["stars"] = evento["result"]["score"]["raw"]
                    else: #Significa que hay un evento de finalizar que no tiene un evento de inicio
                        anomalias.append(evento)
                else:
                    if levelCode in data[name]:
                        data[name][levelCode][-1]["fin"] = None #Porque el jugador sale al menú o cierra el juego
                    else: #Significa que hay un evento de finalizar que no tiene un evento de inicio
                        anomalias.append(evento)
    return data, anomalias              

In [ ]:
#Esta clase sirve para parsear timestamps, recibe 2 y hace la diferencia
class Tiempo:
    def __init__ (self, inicio, final = -1, seconds = -1): #Necesario para tener 2 constructores
        if final == -1: #Primer constructor (1 argumento)
            self.hours = 0
            self.minutes = 0
            self.seconds = 0

            erHours = re.compile(r'\d*h')
            erMinutes = re.compile(r'\d*m')
            erSeconds = re.compile(r'\d*s')
            h = erHours.search(inicio)
            m = erMinutes.search(inicio)
            s = erSeconds.search(inicio)

            if h != None:
                self.hours = h[0][:-1]
            if m != None:
                self.minutes = m[0][:-1]
            if s != None:
                self.seconds = s[0][:-1]

        elif seconds == -1: #Segundo constructor (2 aregumentos)
            self.hours = 0
            self.minutes = 0
            self.seconds = 0

            fin = time.mktime(datetime.datetime.strptime(final, "%Y-%m-%dT%H:%M:%S.%fZ").timetuple())
            ini = time.mktime(datetime.datetime.strptime(inicio, "%Y-%m-%dT%H:%M:%S.%fZ").timetuple())

            hoursRaw = int((fin - ini)/3600)
            minutesRaw = int((fin - ini)/60)
            secondsRaw = int(fin - ini)

            if hoursRaw != 0:
                self.hours = hoursRaw
                minutesRaw = int(((fin - ini)%3600)/60)
            if minutesRaw != 0:
                self.minutes = minutesRaw
                secondsRaw = int((fin - ini)%60)
            self.seconds = secondsRaw
            
        else: #Tercer constructor (3 argumentos)
            self.hours = inicio
            self.minutes = final
            self.seconds = seconds
            
        self.hours = int(self.hours)
        self.minutes = int(self.minutes)
        self.seconds = int(self.seconds)
            
    def toString(self):
        return self.__repr__()

    def __repr__(self):
        t=""
        if self.hours !=0:
            t += str(self.hours) + "h/"
        if self.minutes !=0:
            t += str(self.minutes) + "m/"
        if self.seconds !=0:
            t += str(self.seconds) + "s"
        else:
            t = t[:-1]
        return t
    
    def __add__(self, t): #Sobrecarga del operador +
        s = self.seconds + t.seconds
        m = int(self.minutes) + t.minutes
        h = self.hours + t.hours
        
        if s >= 60:
            s -= 60
            m += 1
        if m >= 60:
            m -= 60
            h += 1
        return Tiempo(h,m,s)
    
    def __truediv__(self, n):
        h = int(self.hours/n)
        self.minutes += (self.hours%n)*60
        m = int(self.minutes/n)
        self.seconds += (self.minutes%n)*60
        s = int(self.seconds/n)
        return Tiempo(h,m,s)

In [ ]:
def tiempoPorNiveles(data, anomalias):
    tiemposJugados = defaultdict(defaultdict)
    for player in data:
        for level in data[player]:
            for times in data[player][level]:
                if times["fin"] != None: #Si no se aborto el intento del nivel
                    try:
                        timeDifference = Tiempo(times["ini"], times["fin"]).toString()
                        if level in tiemposJugados[player]:
                            tiemposJugados[player][level].append({"time" : timeDifference, "stars" : times["stars"]})
                        else:
                            tiemposJugados[player][level] = [{"time" : timeDifference, "stars" : times["stars"]}]
                    except ValueError: #Algunos timestamps de finalizacion estan vacios, ¿Tiene que ser asi?
                        anomalias.append({"jugador" : player,
                                  "nivel" : level,
                                  "Timestamp-Inicio" : times["ini"],
                                  "Timestamp-Fin" : times["fin"],
                                  "Descripcion:" : "No se ha podido parsear uno de los 2 timestamps"})
    return tiemposJugados

In [ ]:
def buscarTarjetas(rawData):
    data = []
    erIdLevel = re.compile(r'/')
    for evento in rawData:
        idJugador = evento["actor"]["name"]
        levelCode = erIdLevel.split(evento["object"]["id"])[-1]
        if idJugador == "bdwyj" and "result" in evento and "extensions" in evento["result"] and "level" in evento["result"]["extensions"]:
            if evento["result"]["extensions"]["level"] == "variables_9":
                data.append(evento)
        elif idJugador == "bdwyj" and levelCode == "variables_9":
            data.append(evento)
    return data

In [ ]:
def getMediaTiemposYEstrellas(data):
    tiemposYEstrellasMedios = defaultdict()
    sumasTotales = defaultdict()
    for user in data:
        for level in data[user]:
            for time in data[user][level]:
                if level in tiemposYEstrellasMedios:
                    tiemposYEstrellasMedios[level]["tiempo"] += Tiempo(time["time"])
                    tiemposYEstrellasMedios[level]["stars"] += time["stars"]
                    sumasTotales[level] += 1
                else:
                    tiemposYEstrellasMedios[level] = {"tiempo" : Tiempo(time["time"]), "stars" : time["stars"]}
                    sumasTotales[level] = 1
    for level in tiemposYEstrellasMedios:
        tiemposYEstrellasMedios[level]["tiempo"] = tiemposYEstrellasMedios[level]["tiempo"]/sumasTotales[level];
        tiemposYEstrellasMedios[level]["stars"] = round(tiemposYEstrellasMedios[level]["stars"]/sumasTotales[level], 2);
    return tiemposYEstrellasMedios

In [ ]:
JSONFile = open('traces_Articoding_Escolapias.json')
rawData = json.load(JSONFile)
data, anomalias = extraerTiempos(rawData)
JSONFile.close()

pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = None
#pd.DataFrame(data)
#dataTarjetas = buscarTarjetas(rawData)
#pd.DataFrame(dataTarjetas)
tiempos = tiempoPorNiveles(data, anomalias)
#pd.DataFrame(tiempos)
tiemposMedios = getMediaTiemposYEstrellas(tiempos)
pd.DataFrame(tiemposMedios)